In [ ]:
import pandas as pd
from io import StringIO
import requests

# URL do dataset na versão numérica
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric'

# Carregar os dados diretamente da URL
# O arquivo não tem cabeçalho, então definimos header=None
# O separador é um espaço
try:
    data = requests.get(url).text
    df = pd.read_csv(StringIO(data), sep='\s+', header=None)
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os dados: {e}")
    # Se a URL não funcionar, você pode tentar carregar o arquivo localmente
    # df = pd.read_csv('caminho/para/german.data-numeric', sep='\s+', header=None)

# O dataset não tem nomes de colunas, então vamos criar uma lista genérica
# O dataset tem 24 colunas de atributos e 1 coluna para o rótulo (target)
col_names = [f'feature_{i}' for i in range(1, 25)] + ['target']
df.columns = col_names

# O dataset original usa 1 para bom crédito e 2 para mau crédito.
# Vamos mudar para 0 (bom) e 1 (mau) para facilitar o treinamento do modelo.
# A coluna 'target' é a última, então a selecionamos.
df['target'] = df['target'].map({1: 0, 2: 1})

# Visualizar as primeiras linhas do dataset
print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

# Visualizar informações sobre o dataset
print("\nInformações do dataset:")
print(df.info())

# Verificar a distribuição da variável 'target'
print("\nDistribuição da variável 'target':")
print(df['target'].value_counts())

import pandas as pd
from io import StringIO
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# --- Passo 1: Carregar e Preparar os Dados (o mesmo código que já usamos) ---
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric'
data = requests.get(url).text
df = pd.read_csv(StringIO(data), sep='\s+', header=None)
col_names = [f'feature_{i}' for i in range(1, 25)] + ['target']
df.columns = col_names
df['target'] = df['target'].map({1: 0, 2: 1})

# --- Passo 2: Separar features e target ---
X = df.drop('target', axis=1)  # Características (features)
y = df['target']              # Variável alvo (target)

# --- Passo 3: Dividir os dados em conjuntos de treino e teste ---
# Vamos usar 70% para treino e 30% para teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Shape dos dados de treino:", X_train.shape, y_train.shape)
print("Shape dos dados de teste:", X_test.shape, y_test.shape)

# --- Passo 4: Treinar o modelo Random Forest ---
# Escolhemos o Random Forest por ser um modelo robusto.
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Passo 5: Avaliar o desempenho do modelo ---
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nAcurácia do modelo: {accuracy:.4f}")

# Instala a biblioteca LIME
!pip install lime

import pandas as pd
from io import StringIO
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import lime
import lime.lime_tabular
import numpy as np

# --- Passo 1: Carregar e Preparar os Dados (reutilizando o código anterior) ---
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric'
data = requests.get(url).text
df = pd.read_csv(StringIO(data), sep='\s+', header=None)
col_names = [f'feature_{i}' for i in range(1, 25)] + ['target']
df.columns = col_names
df['target'] = df['target'].map({1: 0, 2: 1})

X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Passo 2: Criar o Explicador LIME ---
# O LIME precisa de alguns parâmetros do nosso modelo e dos dados
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns.tolist(),
    class_names=['Bom Risco', 'Mau Risco'],
    mode='classification'
)

# --- Passo 3: Escolher um Cliente para Explicar ---
# Vamos escolher o primeiro cliente do conjunto de testes.
# Você pode mudar o índice para analisar outros clientes.
client_index = 0
client_data = X_test.iloc[client_index]
client_target = y_test.iloc[client_index]
client_prediction = model.predict(client_data.values.reshape(1, -1))
print(f"Analisando o cliente com índice {client_index} do conjunto de testes...")
print(f"Previsão do modelo para este cliente: {'Mau Risco' if client_prediction[0] == 1 else 'Bom Risco'}")
print(f"Resultado real: {'Mau Risco' if client_target == 1 else 'Bom Risco'}")

# --- Passo 4: Gerar a Explicação com LIME ---
# Geramos uma explicação para a previsão do cliente escolhido.
explanation = explainer.explain_instance(
    data_row=client_data.values,
    predict_fn=model.predict_proba,
    num_features=5  # Queremos as 5 características mais importantes
)

# --- Passo 5: Visualizar a Explicação ---
# O LIME pode plotar a explicação em um notebook.
print("\nExplicação gerada pelo LIME:")
explanation.show_in_notebook(show_table=True, show_all=False)

# Para salvar a imagem, você pode usar a seguinte função:
# explanation.save_to_file('explicacao_cliente_0.html')